# Averaging models
Probably the best way to ensure robust inferences and estimate errors is to have multiple experimental replicates, ideally on different libraries.

Here we describe how to average model fits across libraries and/or replicates.

## Split data into replicates
We will use our data for the RBD as an earlier examples, but split it into several libraries / replicates.

Specifically, we will fit two different libraries: `avg2muts` and `avg3muts`, which have different barcodes and also different mutation rates (although of course in real life you might sometimes want to average results from different libraries with the same mutation rates).
We will also simulate having two replicates for each library just by sampling each library.
To make this example faster, we'll just use one concentration:

In [1]:
import pandas as pd

import polyclonal.polyclonal
import polyclonal.polyclonal_collection


# read data
all_data = pd.read_csv("RBD_variants_escape_noisy.csv", na_filter=None)

# split by library and replicates
libraries = ["avg2muts", "avg3muts"]  # the two libraries to use
concentrations = [1]  # use just use this concentration
n_replicates = 2  # number of replicates per library

data_by_replicate = {
    (library, replicate + 1): (
        all_data.query("library == @library")
        .query("concentration in @concentrations")
        .sample(frac=0.3, random_state=replicate)
    )
    for library in libraries
    for replicate in range(n_replicates)
}

## Fit models to each replicate
We now fit a `Polyclonal` model to each replicate using just 2 epitopes, as the data don't seem sufficient to accurately fit all three epitopes.
Then we arrange the models in a data frame:

In [2]:
# first create a data frame with all the models
models_by_replicate = {}
for (library, replicate), data in data_by_replicate.items():
    model = polyclonal.Polyclonal(data_to_fit=data, n_epitopes=2)
    models_by_replicate[(library, replicate)] = model
models_df = (
    pd.Series(models_by_replicate, name="model")
    .rename_axis(["library", "replicate"])
    .reset_index()
)

# now fit the models
n_fit, n_failed, models_df["model"] = polyclonal.polyclonal_collection.fit_models(
    models_df["model"],
    n_threads=2,
    reg_escape_weight=0.01,
    reg_uniqueness2_weight=0,
)

Note how the models are arranged in a data frame:

In [3]:
# NBVAL_IGNORE_OUTPUT

models_df

,library,replicate,model
0,avg2muts,1,<polyclonal.polyclonal.Polyclonal object at 0x...
1,avg2muts,2,<polyclonal.polyclonal.Polyclonal object at 0x...
2,avg3muts,1,<polyclonal.polyclonal.Polyclonal object at 0x...
3,avg3muts,2,<polyclonal.polyclonal.Polyclonal object at 0x...


## Average the models
Now we create a `PolyclonalAverage` model with the models to average.
Note that by default the "average" used by `PolyclonalAverage` is the **median** rather than the **mean** between epitopes, although this is a parameter that can also be set to mean.

If your epitopes are too different or poorly defined (e.g., you are trying to fit more epitopes than can be consistently inferred from the data), then you may get an epitope harmonization error:

In [4]:
model_avg = polyclonal.PolyclonalAverage(models_df)

Let's look at the correlation among the escape at each epitope across models:

In [5]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_escape_corr_heatmap()

alt.Chart(...)

Look at the activities of the epitopes and the rest of the curves.
Note how a dark line is shown for the average, and thin lines for individual replicates.
It should generally be the case that the epitope with greater activity (more left shifted in plot below) should also be better correlated among replicates (heatmap above) as it can be inferred more reliably:

In [6]:
# NBVAL_IGNORE_OUTPUT

model_avg.curves_plot()

alt.Chart(...)

We can access the average escape values:

In [7]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_escape_df

,epitope,site,wildtype,mutant,mutation,escape_mean,escape_median,escape_std,n_models,times_seen,frac_models
0,1,331,N,A,N331A,-0.025324,-0.009349,0.334581,4,17.75,1.0
1,1,331,N,D,N331D,-0.084136,-0.074723,0.033504,4,11.25,1.0
2,1,331,N,E,N331E,0.001074,0.006871,0.017662,4,10.25,1.0
3,1,331,N,F,N331F,0.284670,0.165276,0.319248,4,10.00,1.0
4,1,331,N,G,N331G,0.176953,0.102155,0.226222,4,25.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
3859,2,531,T,R,T531R,-0.164549,-0.091317,0.175319,4,27.00,1.0
3860,2,531,T,S,T531S,-0.092637,-0.009036,0.177302,4,31.75,1.0
3861,2,531,T,V,T531V,-0.044969,-0.015454,0.099601,4,19.50,1.0
3862,2,531,T,W,T531W,0.172431,0.103343,0.268597,4,5.25,1.0


Or the per-replicate escape values:

In [8]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_escape_df_replicates

,epitope,site,wildtype,mutant,mutation,escape,times_seen,library,replicate
0,1,331,N,A,N331A,-0.061598,19,avg2muts,1
1,1,331,N,D,N331D,-0.129411,10,avg2muts,1
2,1,331,N,E,N331E,-0.000172,11,avg2muts,1
3,1,331,N,F,N331F,0.055094,10,avg2muts,1
4,1,331,N,G,N331G,0.046051,18,avg2muts,1
...,...,...,...,...,...,...,...,...,...
15429,2,531,T,R,T531R,-0.057570,24,avg3muts,2
15430,2,531,T,S,T531S,-0.015650,42,avg3muts,2
15431,2,531,T,V,T531V,-0.030629,24,avg3muts,2
15432,2,531,T,W,T531W,0.537803,6,avg3muts,2


Now let's plot the escape.
See how you can select mutations based not only on how many times they are seen (averaged over all models in average), but also the number of models in which they are seen.

In [9]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_escape_plot(addtl_slider_stats={"times_seen": 2})

alt.VConcatChart(...)